# Groupe Relative Policy Optimization (GRPO)

Install the Hugging Face libraries to run this notebook.

In [10]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\soelm\Documents\04_Code\MVA\MVA_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Your goal is to fill in the `GRPOTrainer` class. You have two options (and you can do both):
* the "normal GRPO" with clipped surrogate objective
* or the "vanilla GRPO" with original objective

In [30]:
questions = [
    "A train takes 3 hours to travel from A to B at an average speed of 60 km/h. How long would the trip take if the train traveled at 80 km/h?",
    "A snail climbs a 10-meter wall. It climbs 3 meters during the day and slips 2 meters at night. How many days will it take to reach the top?",
    'If a liar says, "I always lie," is he telling the truth?',
    'Can we say that "this sentence is false"? Explain why.',
    "Paul is twice the age Pierre was when Paul was the age Pierre is today. If Pierre is 20 years old, how old is Paul?",
    "A father and his son together are 36 years old. The father is exactly three times the son's age. How old is the son?",
    "All the cats I have met so far were black. Can I conclude that all cats are black? Why?",
    "If all humans are mortal and Socrates is human, what can we conclude?",
    "If a shirt costs twice as much as a pair of pants and the pants cost 30€, how much does the shirt cost?",
    'Jean says: "All my friends are football players." Pierre is Jean’s friend. Can we conclude that Pierre is a football player?',
    "You are in a train and must choose between switching the direction of the train to avoid five people tied to one track, but in doing so, you will kill one person on the other track. What do you do and why?",
    "A doctor has five patients in need of organ transplants, and a perfectly healthy patient comes in for a routine check-up. Should the doctor sacrifice this patient to save the five others?",
    "What would happen if gravity on Earth were twice as strong?",
    "If humans could read minds, how would that change society?"
]

dataset = []
for question in questions:
    dataset.append(f"A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> <\answer> tags, respectively, i.e., <think> reasoning process here <\think> <answer> answer here <\answer>. User: {question}. Assistant:")

In [103]:
class GRPOConfiguration:
    def __init__(self,
                 model_name="gpt2",
                 learning_rate=1e-5,
                 temperature=0.9,
                 max_prompt_length=200,
                 max_output_length=200,
                 device="cpu",
                 num_generations=3,
                 num_iterations=2,
                 beta=0.1,
                 epsilon=1e-5,
                 reward_func=None,
                 print_outputs=False,
                 print_advantages=False):
        
        self.model_name = model_name
        self.learning_rate = learning_rate
        self.temperature = temperature
        self.max_prompt_length = max_prompt_length
        self.max_output_length = max_output_length
        self.device = device
        self.num_generations = num_generations # number of generations per prompts
        self.num_iterations = num_iterations    # number of iterative optimization steps per prompts
        self.beta = beta # KL coefficient
        self.epsilon = epsilon
        self.reward_func = reward_func # reward function
        self.print_outputs = print_outputs
        self.print_advantages = print_advantages


In [111]:
#Inspired from HuggingFace
import re
import copy


class GRPOTrainer:
    def __init__(self, config: GRPOConfiguration):
        """
        Initialize a GRPO Trainer
        Args:
            config: GRPO Configuration
        """
        self.device = config.device
        self.model_name = config.model_name
        self.model = AutoModelForCausalLM.from_pretrained(config.model_name).to(self.device) # model to optimize
        self.ref_model = None # reference model
        self.tokenizer = AutoTokenizer.from_pretrained(config.model_name, padding=True, padding_side="left")
        self.tokenizer.pad_token = self.tokenizer.eos_token  #gpt2 n'a pas de padding token
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.learning_rate)
        self.temperature = config.temperature
        self.max_prompt_length = config.max_prompt_length
        self.max_output_length = config.max_output_length
        self.eps = config.epsilon
        self.num_generations = config.num_generations # num of generation per prompts
        self.num_iterations = config.num_generations
        self.beta = config.beta
        self.reward_func = config.reward_func if config.reward_func else self._default_reward_func
        self.print_outputs = config.print_outputs
        self.print_adantages = config.print_advantages

    
    def _default_reward_func(self, prompt, outputs, **kwargs):
        """Reward function that checks if the completion has a specific format."""
        
        pattern = r"^<think>.*?</think><answer>.*?</answer>$"

        matches = [re.match(pattern, content) for content in outputs]
        return [1.0 if match else 0.0 for match in matches]
        

    def _get_per_token_logps(self, model, input_ids, attention_mask, logits_to_keep):
        """
        Get the per-token log propabilities for the outputs.
        Args:
            model: model to compute per-token log probabilities
            input_ids: sequence of tokens #(G, L)
            attention_mak: mapping of tokens to keep to compute attention (excluding padding)
            logits_to_keep: number of logits to keep to compute the loss
        """
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        logits = logits[:,:-1,:] # (G, L-1, V) exclude the last logit corresponding to next token prediction

        input_ids = input_ids[:, -logits_to_keep:]
        logits = logits[:, -logits_to_keep:]
        log_probs = logits.log_softmax(-1) # softmax to get probabilities
        return torch.gather(log_probs, dim=-1, index=input_ids.unsqueeze(-1)).squeeze(-1) # keeping only input_ids
    

    def _generate_and_score_outputs(self, prompt: str):
        """
        Generate answers with old_model to the prompt, compute associated reward and advantages and those of ref_model.
        Args:
            prompt: the prompt
        """
        device = self.device

        # Tokenization
        prompt_inputs = self.tokenizer(prompt, return_tensors="pt", padding=False).to(device)
        prompt_ids, prompt_mask = prompt_inputs["input_ids"], prompt_inputs["attention_mask"]

        # Setting to prompt lenght
        if self.max_prompt_length is not None:
            prompt_ids = prompt_ids[:, -self.max_prompt_length :]
            prompt_mask = prompt_mask[:, -self.max_prompt_length :]

        # Generate answers with old_model
        with torch.inference_mode():
            prompt_output_ids = self.model.generate(
                prompt_ids,
                attention_mask=prompt_mask,
                num_return_sequences=self.num_generations, # number of generations
                do_sample=True,
                max_length=prompt_ids.size(1) + self.max_output_length,  # (P+O)
                eos_token_id=self.tokenizer.eos_token_id
            )

        if self.print_outputs:
            for i in range(prompt_output_ids.size(0)):
                decoded_sequence = self.tokenizer.convert_ids_to_tokens(prompt_output_ids[i, :].tolist())
                print(f"Réponse générée (génération {i}) de len {len(decoded_sequence)}: {' '.join(decoded_sequence)}")

        # Splitting prompt and outputs tokens
        prompt_length = prompt_ids.size(1)
        prompt_ids = prompt_output_ids[:, :prompt_length]  # (G, P)
        output_ids = prompt_output_ids[:, prompt_length:]  # (G, O)


        # Padding with EOS after first EOS in outputs
        is_eos = output_ids == self.tokenizer.eos_token_id  # (G, O)
        eos_idx = torch.full((is_eos.size(0),), is_eos.size(1), dtype=torch.long, device=device)  # (G,)
        has_eos = is_eos.any(dim=1)  # (G,)
        first_eos_idx = is_eos.int().argmax(dim=1)  # Index of first EOS token (G,)
        eos_idx[has_eos] = first_eos_idx[has_eos]  # (G,)
        sequence_indices = torch.arange(is_eos.size(1), device=device).expand_as(is_eos)  # (G, O)
        output_mask = (sequence_indices <= eos_idx.unsqueeze(1)).int()  # (G, O) Mask for padding

        # Merging prompt and output  mask
        attention_mask = torch.cat([prompt_mask.expand(self.num_generations, -1), output_mask], dim=1)  # (G, P+O)

        logits_to_keep = output_ids.size(1) # Logits to keep for loss computation

        # Compute log-probabilities for the prompt with model_ref and model_old
        self.model.eval()
        with torch.inference_mode():
            # When using num_iterations == 1, old_per_token_logps == per_token_logps
            if self.num_iterations > 1:
                old_per_token_logps = self._get_per_token_logps(self.model, prompt_output_ids, attention_mask, logits_to_keep)
            else:
                old_per_token_logps = None

            ref_per_token_logps = self._get_per_token_logps(self.ref_model, prompt_output_ids, attention_mask, logits_to_keep)
        
        # Decoding outputs
        outputs = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)

        # Compute rewards
        output_rewards = self.reward_func(prompt=prompt, outputs=outputs)
        rewards = torch.tensor(output_rewards, dtype=torch.float32, device=self.device) #(G,)

        # Compute advantages
        mean_grouped_rewards = rewards.view(-1, self.num_generations).mean(dim=1)
        mean_grouped_rewards = mean_grouped_rewards.repeat_interleave(self.num_generations, dim=0)
        std_grouped_rewards = rewards.view(-1, self.num_generations).std(dim=1)
        std_grouped_rewards = std_grouped_rewards.repeat_interleave(self.num_generations, dim=0)

        advantages = (rewards - mean_grouped_rewards) / (std_grouped_rewards + 1e-4) # (G,)

        return {
            "prompt_ids": prompt_ids,
            "prompt_mask": prompt_mask,
            "output_ids": output_ids,
            "output_mask": output_mask,
            "old_per_token_logps": old_per_token_logps, 
            "ref_per_token_logps": ref_per_token_logps,
            "advantages": advantages,}



    def compute_loss(self, model, inputs):
        """
        Compute Loss according to GRPO paper, using advantages, per_token probabilities and KL divergence approximator to reference model
        Args:
            model: enlever
            inputs: {"prompt_ids", "prompt_mask", "output_ids", "output_mask", "old_per_token_logps", "ref_per_token_logps", "advantages"}
        """

        # Compute the per-token log probabilities for the current model
        prompt_ids, prompt_mask = inputs["prompt_ids"], inputs["prompt_mask"]
        output_ids, output_mask = inputs["output_ids"], inputs["output_mask"]
        input_ids = torch.cat([prompt_ids, output_ids], dim=1)
        
        prompt_mask = prompt_mask.expand(output_mask.size(0), -1)
        attention_mask = torch.cat([prompt_mask, output_mask], dim=1)
        logits_to_keep = output_ids.size(1)  # we only need to compute the logits for the completion tokens

        per_token_logps = self._get_per_token_logps(model, input_ids, attention_mask, logits_to_keep) # policy of current model

        # Compute the KL divergence between the current model and the reference model
        if self.beta != 0.0:
            ref_per_token_logps = inputs["ref_per_token_logps"]
            per_token_kl = (torch.exp(ref_per_token_logps - per_token_logps) - (ref_per_token_logps - per_token_logps) - 1 ) # (G, O) KL differentiable approximator

        ### Compute the loss
        
        advantages = inputs["advantages"]
        if self.print_adantages:
            print( "advantages :", advantages)

        # When using num_iterations == 1, old_per_token_logps == per_token_logps
        old_per_token_logps = inputs["old_per_token_logps"] if self.num_iterations > 1 else per_token_logps.detach()
        policy_ratio = torch.exp(per_token_logps - old_per_token_logps) # (G, O)
        policy_ratio_clipped = torch.clamp(policy_ratio, 1 - self.eps, 1 + self.eps) #clipped ratio

        per_token_loss1 = policy_ratio * advantages.unsqueeze(1)
        per_token_loss2 = policy_ratio_clipped * advantages.unsqueeze(1)
        per_token_loss = -torch.min(per_token_loss1, per_token_loss2)

        if self.beta != 0.0:
            per_token_loss = per_token_loss + self.beta * per_token_kl # adding KL

        loss = (per_token_loss * output_mask).sum() / output_mask.sum() # excluding padding in loss computation

        return loss
    

    def train(self, num_epochs: int, dataset: list):
        """
        Train the model on the dataset for specific number of epochs.
        Args:
            num_epochs: number of epochs 
            dataset: dataset
        """
        self.ref_model = copy.deepcopy(self.model)  # Initialize reference model

        for epoch in range(num_epochs):
            total_loss = 0
            for i, prompt in enumerate(dataset):
                print(f"Training on prompt {i}")
                loss = self.train_step(prompt)
                total_loss += loss
            
            avg_loss = total_loss / len(dataset)
            print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}") 
            self.ref_model = copy.deepcopy(self.model)


    def train_step(self, prompt: str):
        """
            Do a traing step on a specific prompt with multiple iteration steps.
            Args:
                prompt: prompt to optimize the model on it
        """
        
        self.model.train()

        # Generate outputs with old_model, compute log-probabilities for old_model and ref_model
        inputs = self._generate_and_score_outputs(prompt)

        # For self.num_iterations, iterativelt update the model on the same prompt
        for i in range(1, self.num_iterations): 
            
            print(f"GRPO Iteration {i}")

            loss = self.compute_loss(self.model, inputs)

            # Backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        return loss.item()



In [113]:
def short_reward_func(prompt, outputs, **kwargs):
    """Reward function that gives higher scores to longer completions."""
    print(type(outputs))
    return [float(len(completion)) for completion in outputs]

dataset = ["The sky is", "Yesterday, I went to", "I hope I will", "My car is", "I have to go to", "My friend want"]


config = GRPOConfiguration(num_generations=3, num_iterations=4, print_outputs=True, print_advantages=True, reward_func=short_reward_func)
trainer = GRPOTrainer(config)
trainer.train(3, dataset)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Training on prompt 0
Réponse générée (génération 0) de len 203: The Ġsky Ġis Ġa Ġmess ." Ċ Ċ The Ġman Ġhas Ġmore Ġthan Ġ10 Ġyears Ġin Ġthe Ġnews Ġbusiness Ġand Ġhe Ġhas Ġno Ġdesire Ġto Ġbe Ġseen Ġof Ġany . ĠHe Ġis Ġonly Ġlooking Ġat Ġhis Ġjob , Ġthe Ġvery Ġvery Ġnext Ġday . Ċ Ċ You Ġsee Ġit Ġwhen Ġyou Ġsee Ġit ... Ċ Ċ Ċ " It 's Ġabout Ġthe Ġway Ġthat Ġit 's Ġdone ..." Ċ Ċ As Ġyou Ġwatch Ġhim Ġmake Ġone Ġfinal Ġmove , Ġyou Ġrealize Ġthat Ġhe Ġis Ġonly Ġhalf Ġright Ġand Ġthat Ġthat Ġfact Ġis Ġall Ġyou Ġcan Ġsee Ġbut Ġyou Ġcan 't Ġquite Ġsee Ġwhat 's Ġbeing Ġreported . Ċ Ċ The Ġsky Ġis Ġactually Ġa Ġvery Ġbusy Ġsky Ġon Ġyour Ġown Ġand Ġall Ġyou Ġcan Ġsee Ġis Ġthat Ġpart Ġof Ġwhat Ġyou Ġwish Ġyou Ġcould Ġunderstand . Ċ Ċ The Ġworld Ġwe Ġsee Ġis Ġa Ġbig Ġmess Ġin Ġthe Ġworld Ġof Ġnews Ġthat Ġis Ġonly Ġhalf Ġof Ġthe Ġworld Ġyou Ġcould Ġsee Ġat Ġthe Ġmoment . ĠAs Ġa Ġlittle Ġkid Ġyou Ġthink Ġabout Ġit Ġas Ġthey Ġsee . ĠBut Ġyou Ġdon 't Ġsee Ġthem Ġbecause Ġyou Ġonly Ġsaw Ġthem Ġin Ġpictures .

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Training on prompt 1
Réponse générée (génération 0) de len 205: Yesterday , ĠI Ġwent Ġto Ġthe Ġgym Ġwith Ġsome Ġfriends Ġthat Ġday , Ġit Ġwas Ġgoing , Ġ' I 'm Ġgoing Ġto Ġgo Ġto Ġher '. ĠTo Ġknow Ġher Ġso Ġfar , Ġshe 's Ġbeen Ġvery Ġstrong , Ġshe 's Ġnot Ġa Ġreal Ġhot Ġchick , Ġwho ĠI Ġgot Ġto Ġhave Ġit Ġwith , Ġeven Ġthough ĠI Ġdon 't Ġget Ġto Ġhave Ġmany Ġfriends , Ġshe Ġwas Ġa Ġmuch Ġbetter Ġperson Ġwith Ġme Ġthat Ġnight , Ġso Ġthat 's Ġa Ġgood Ġthing , Ġthat 's Ġwhat ĠI Ġexpected . ĊĊ Ċ But Ġyeah , ĠI Ġgot Ġlucky Ġon Ġthe Ġlast Ġday , ĠI Ġget Ġlucky Ġon Ġthe Ġlast Ġday , ĠI Ġwent Ġto Ġthe Ġgym Ġwith Ġa Ġfriend , ĠI Ġthink Ġit Ġwas Ġprobably Ġthe Ġsame Ġgirl Ġyou Ġgot Ġto Ġhave Ġa Ġlittle Ġbit Ġof Ġexcitement Ġfor Ġme Ġthat Ġthe Ġday Ġthat ĠI Ġcame Ġfor Ġme ? ĊĊ Ċ D erek ĊĊ Ċ [ 1 - 04 - 15 ] ĠI Ġfeel Ġbad Ġfor Ġmyself Ġbecause ĠI Ġdon 't Ġknow , ĠI Ġjust Ġcouldn 't . ĠI Ġthink ĠI Ġmade Ġfun Ġof Ġthe Ġworld . ĠIt Ġreally , Ġit 's Ġall Ġthat . ĠAnd , ĠI Ġthink Ġmaybe Ġthat 's Ġa Ġlitt

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Training on prompt 3
Réponse générée (génération 0) de len 203: My Ġcar Ġis Ġin Ġit 's Ġplace . ĠIt Ġis Ġnot Ġyour Ġcar . ĠIt Ġis Ġnot Ġyour Ġcar 's Ġplace . ĠIt Ġis Ġnot Ġsome Ġcar Ġfrom Ġyou Ġsay Ġthere Ġyou Ġare . ĠIt Ġis Ġa Ġthing Ġyou Ġtell Ġme Ġyou Ġdo Ġsee Ġin Ġyour Ġroom . ĠYour Ġroom . ĠAnd Ġjust Ġbecause Ġyou Ġare Ġstill Ġyou Ġin Ġa Ġcar Ġthere Ġdoesn 't Ġmean Ġyou Ġare Ġnot Ġthere . ĠThis Ġis Ġsome Ġplace . ĠThey Ġare Ġlike Ġa Ġlittle Ġplace Ġyou Ġknow Ġit Ġyou Ġare Ġright Ġthere Ġright Ġnow Ġdon 't Ġthink Ġabout Ġyour Ġroom . ĠI Ġdon 't Ġknow Ġthis , Ġthis Ġis Ġsomething Ġyou Ġwill Ġthink Ġabout Ġthis . ĠAnd Ġif Ġyou Ġgo Ġto Ġyour Ġcar Ġyou Ġcan Ġbe Ġthere . ĠYour Ġcar Ġyou Ġknow Ġwhat ? ĠAnd Ġtell Ġme Ġthen Ġtell Ġme Ġthat Ġyou Ġare Ġthere . ĠYour Ġcar . ĠYour Ġcar Ġon Ġyour Ġway . ĠThe Ġcar . ĠI Ġtold Ġyou Ġin Ġmy Ġroom Ġthat Ġis Ġright Ġway Ġthen Ġit 's Ġa Ġlittle Ġplace . ĠIt Ġis Ġnothing Ġmore Ġthan Ġthat . ĠIt Ġis Ġjust Ġsomething Ġthat Ġwhen ĠI Ġwas Ġwalking Ġmy Ġcar

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Training on prompt 4
Réponse générée (génération 0) de len 205: I Ġhave Ġto Ġgo Ġto Ġwhat 's Ġhappening , Ġthe Ġproblem Ġis Ġthat Ġno Ġtime Ġhas Ġpassed Ġand Ġthere Ġaren 't Ġany Ġattempts Ġwere Ġmade . ĠThis Ġis Ġthe Ġbig Ġone Ġfor Ġme Ġand ĠI 've Ġbeen Ġasking Ġfor Ġit , Ġthe Ġissue Ġis Ġalso ĠI Ġwill Ġsay Ġwhat Ġthe Ġprocess Ġof Ġthe Ġissue Ġis Ġand ĠI Ġwill Ġdo Ġwhat Ġhas . Ċ Ċ I Ġhave Ġdone Ġall Ġthat ĠI Ġneed Ġto Ġknow Ġa Ġlot Ġfrom Ġthe Ġissues Ġthat Ġare Ġin Ġthe Ġissues Ġthat Ġwe Ġneed . <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|> <|endoft

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Training on prompt 5
Réponse générée (génération 0) de len 203: My Ġfriend Ġwant Ġto Ġplay Ġwith Ġme Ġand ĠI Ġplay Ġwith Ġmy Ġpartner . ĠWe Ġshould Ġthink Ġabout Ġeach Ġother 's Ġminds ." Ċ Ċ In Ġthis Ġcontext , Ġ" you " Ġis Ġnot Ġa Ġcompliment , Ġbut Ġa Ġname , Ġa Ġname , Ġnot Ġin Ġthe Ġleast , Ġor Ġa Ġmoniker Ġthat Ġis Ġsomething Ġas Ġbig Ġas Ġsomething Ġof Ġsomething Ġwhich Ġyou Ġare Ġvery Ġcapable Ġof Ġthinking , Ġand Ġhow Ġmany Ġof Ġyou Ġare Ġcapable Ġof Ġthinking . ĠA Ġcertain Ġtype Ġof Ġname Ġwhich Ġis Ġan Ġindividual Ġname Ġas Ġa Ġsingular Ġname , Ġand Ġthere Ġis Ġa Ġcertain Ġtype Ġof Ġtype Ġof Ġnoun Ġwhich Ġis Ġin Ġa Ġsingular Ġor Ġplural Ġway , Ġand Ġas Ġthey Ġlike Ġto Ġbe Ġand Ġin Ġthis Ġcategory Ġwhere Ġit Ġis Ġan Ġexact Ġtype Ġof Ġnoun Ġ( it 's Ġonly Ġin Ġa Ġsingular Ġor Ġplural ) Ġthat Ġis Ġin Ġa Ġsingular Ġor Ġplural Ġway Ġof Ġsaying , Ġor Ġsomething Ġin Ġthis Ġis Ġa Ġsingular Ġor Ġplural Ġor Ġa Ġspecific Ġtime . ĠFor Ġexample , Ġif Ġyou Ġlike Ġto Ġwrite Ġthat ĠI 'm Ġtal